In [7]:
%pip install -U nltk

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.0 MB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -U summac

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 181.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 47.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 53.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 47.3 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 56.7 MB/s eta 0:00:00
INFO: pip is still looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os

# 你的 nltk_data 解压目录（改成你真实路径）
os.environ["NLTK_DATA"] = "/home/dataset-assist-0/nltk_data"

# HuggingFace 离线（防止任何联网）
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

# HF_HOME 可选：如果你想让缓存都在一个地方（不设也行）
# os.environ["HF_HOME"] = "/home/dataset-assist-0/.cache/huggingface"


In [7]:
import nltk
nltk.data.path.append(os.environ["NLTK_DATA"])

# 两个都不报错才算ok
nltk.data.find("tokenizers/punkt")
nltk.data.find("tokenizers/punkt_tab/english")
print("NLTK punkt/punkt_tab OK")


NLTK punkt/punkt_tab OK


In [8]:
import torch
from summac.model_summac import SummaCConv, model_map

# ① 这里填：包含 config.json 的那一层目录
VITC_DIR = "/home/dataset-assist-0/eval_assets/summac/tals_albert-xlarge-vitaminc-mnli"

# ② 这里填：summac conv 权重文件路径
SUMMAC_BIN = "/home/dataset-assist-0/summac_conv_vitc_sent_perc_e.bin"

# 关键：把 vitc 的 model_card 改成“本地目录”，否则会尝试去HF联网
model_map["vitc"]["model_card"] = VITC_DIR

summac_model = SummaCConv(
    models=["vitc"],
    bins="percentile",
    granularity="sentence",
    nli_labels="e",
    device="cuda" if torch.cuda.is_available() else "cpu",
    start_file=SUMMAC_BIN,
    agg="mean",
    max_doc_sents=200,
)

print("SummaC model ready:", type(summac_model))


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


<All keys matched successfully>
SummaC model ready: <class 'summac.model_summac.SummaCConv'>


In [9]:
doc = "NVIDIA reported revenue of $10 billion in fiscal year 20XX."
summary = "The company reported $10 billion revenue."

out = summac_model.score([doc], [summary])
print(out)


/opt/conda/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


{'scores': [0.592324435710907]}


/opt/conda/lib/python3.11/site-packages/summac/model_summac.py:298: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  histograms = torch.FloatTensor(histograms).to(self.device)


In [10]:
from pathlib import Path
import json, time

REPORT_DIR = Path("/home/dataset-assist-0/annual-reports_output")
PDF_DIR    = Path("/home/dataset-assist-0/annual-reports")

OUT_JSONL  = REPORT_DIR / "summac_scores.jsonl"
OUT_CSV    = REPORT_DIR / "summac_scores.csv"

print("reports:", REPORT_DIR)
print("pdfs:", PDF_DIR)


reports: /home/dataset-assist-0/annual-reports_output
pdfs: /home/dataset-assist-0/annual-reports


In [11]:
def pdf_to_text(pdf_path: str) -> str:
    # 方案A：PyMuPDF（fitz）
    try:
        import fitz  # pymupdf
        doc = fitz.open(pdf_path)
        parts = []
        for page in doc:
            t = page.get_text("text")
            if t:
                parts.append(t)
        return "\n\n".join(parts)
    except Exception as e:
        # 方案B：pdfplumber（如果你装了）
        try:
            import pdfplumber
            parts = []
            with pdfplumber.open(pdf_path) as pdf:
                for p in pdf.pages:
                    t = p.extract_text() or ""
                    if t:
                        parts.append(t)
            return "\n\n".join(parts)
        except Exception as e2:
            raise RuntimeError(f"PDF parse failed: {pdf_path}\nfitz err={e}\npdfplumber err={e2}")


In [12]:
def report_to_pdf_path(report_txt_path: Path) -> Path:
    # e.g. NVIDIA-2019-Annual-Report_short_report.txt -> NVIDIA-2019-Annual-Report.pdf
    base = report_txt_path.stem
    if not base.endswith("_short_report"):
        raise ValueError(f"Unexpected report name: {report_txt_path.name}")
    pdf_stem = base[:-len("_short_report")]
    return PDF_DIR / f"{pdf_stem}.pdf"


In [13]:
from tqdm.auto import tqdm

# 你已经在上面初始化好了 summac，这里直接用
# summac.score([source], [report])  -> dict

pdf_text_cache = {}  # pdf_path -> text

def safe_read_text(p: Path) -> str:
    return p.read_text(encoding="utf-8", errors="ignore")

def pick_score(res):
    # 兼容不同 summac 返回结构
    if isinstance(res, dict):
        if "scores" in res and res["scores"]:
            return float(res["scores"][0])
        if "score" in res:
            return float(res["score"])
    return None

rows = []
report_files = sorted(REPORT_DIR.glob("*_short_report.txt"))
print("found report files:", len(report_files))

for rpt in tqdm(report_files):
    pdf = report_to_pdf_path(rpt)

    if not pdf.exists():
        rows.append({
            "report_file": rpt.name,
            "pdf_file": pdf.name,
            "status": "pdf_missing",
            "summac_vitc": None
        })
        continue

    report = safe_read_text(rpt).strip()
    if not report:
        rows.append({
            "report_file": rpt.name,
            "pdf_file": pdf.name,
            "status": "empty_report",
            "summac_vitc": None
        })
        continue

    # 取/算 source_text（年报原文）
    if pdf not in pdf_text_cache:
        txt = pdf_to_text(str(pdf))
        pdf_text_cache[pdf] = txt
    annual_text = pdf_text_cache[pdf]

    # 为了速度和稳定性：只取前 200k 字符（你也可以调大/调小）
    source = annual_text[:200_000]

    t0 = time.time()
    res = summac_model.score([source], [report])
    score = pick_score(res)
    dt = time.time() - t0

    rows.append({
        "report_file": rpt.name,
        "pdf_file": pdf.name,
        "status": "ok",
        "summac_vitc": score,
        "source_chars": len(source),
        "report_chars": len(report),
        "eval_seconds": round(dt, 3),
    })

print("done. ok:", sum(r["status"]=="ok" for r in rows), "total:", len(rows))


found report files: 8


100%|██████████| 8/8 [02:32<00:00, 19.10s/it]

done. ok: 8 total: 8


In [14]:
from pathlib import Path
import json, csv

# ① 你想保存到哪里，就改这个目录
SAVE_DIR = Path("/home/dataset-assist-0/eval_results")   # 例子：新目录
SAVE_DIR.mkdir(parents=True, exist_ok=True)             # 自动创建目录

# ② 输出文件名（你也可以改名字）
OUT_JSONL = SAVE_DIR / "summac_scores.jsonl"
OUT_CSV   = SAVE_DIR / "summac_scores.csv"

# ③ 写 jsonl
OUT_JSONL.write_text(
    "\n".join(json.dumps(r, ensure_ascii=False) for r in rows),
    encoding="utf-8"
)
print("saved:", OUT_JSONL)

# ④ 写 csv（不依赖 pandas）
with OUT_CSV.open("w", newline="", encoding="utf-8") as f:
    w = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
    w.writeheader()
    w.writerows(rows)
print("saved:", OUT_CSV)


saved: /home/dataset-assist-0/eval_results/summac_scores.jsonl
saved: /home/dataset-assist-0/eval_results/summac_scores.csv
